## LSTM IMPLEMENTATION KERAS EXPLANATION


TO BEGIN ALSO SEE A DETAILED EXPLANATION OF KERAS EMBEDDING LAYER.

#### IMPORTING MODULES

In [2]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
% matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#import nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# BeautifulSoup libraray
from bs4 import BeautifulSoup 

import re # regex

#keras
import keras
from keras.preprocessing.text import one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Flatten ,Embedding,Input,Dropout,LSTM,CuDNNLSTM
from keras.models import Model
from keras.utils import to_categorical

#model selection
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


#### CREATING A SAMPLE CORPUS

Below I am creating a sampple corpus of just 3 documents which should be enough to depict the working of keras embedding layer
as well as the implementation of lstm in Keras librray. 

**Do not focus on the accurcay while training as the dtaa is just purely random(bas aise hi kuch bhi utha liya hai ki chal jaye main purpose lstm ko implement karna hai)**

In [0]:
sample_text_1="bitty bought a bit of butter"
sample_text_2="but the bit of butter was a bit bitter"
sample_text_3="so she bought some better butter to make the bitter butter better"

corp=[sample_text_1,sample_text_2,sample_text_3]
no_docs=len(corp)

#### INTEGER ENCODING THE DOCUMENTS

In [6]:
vocab_size=50 
encod_corp=[]
for i,doc in enumerate(corp):
    encod_corp.append(one_hot(doc,50))
    print("The encoding for document",i+1," is : ",one_hot(doc,50))

The encoding for document 1  is :  [5, 14, 17, 6, 35, 18]
The encoding for document 2  is :  [46, 48, 6, 35, 18, 6, 17, 6, 47]
The encoding for document 3  is :  [49, 20, 14, 17, 33, 18, 28, 2, 48, 47, 18, 33]


In [7]:
maxlen=-1
for doc in corp:
    tokens=nltk.word_tokenize(doc)
    if(maxlen<len(tokens)):
        maxlen=len(tokens)
print("The maximum number of words in any document is : ",maxlen)

The maximum number of words in any document is :  12


### PADDING THE DOCUMENTS SO THAT ALL ARE OF SAME LENGTH AS REQD BY KERAS EMBEDDING LAYER

**Note that there is one way to work with variable lenght reviews in keraslike b suing some 'Masking' layer or by specifying 'NOne' in place of 'max_doc_len' in 'input_lenght' in Keras or in 'Input(shape=(None,))' when using function api of keras.
but ye tarike ko sai se karna kaise hai that is not very clear to me at the moment.**

**Also one approach widely mentioned on the net is that different batches while training (batch_size that we specify) can have different lengths although obervations (or text as dealing with text classification or reviews in sentiment analysis) within same batch should have same size.(like all reviews in same batch should have same no of words). so what we can do is specify batch_size while training equal to 1 but that is too inefficient.**

**to improve upon batch_size of 1 what we can do is to group observations with same text in same batches (like say for sentiment analysis ke liye reviews ko no of words ke hissab se sort kar loo and then jinme same no of woeds ho unkko same batchesme daal doo). bUt ye traika thoda ajeeb lagra hai and isliye abhi nai kiya hai though [iss](https://adventuresinmachinelearning.com/keras-lstm-tutorial/) blog me SHAYAD aisa hi kuch kiy ahai.**


**Last option is to pad the reviews or texts and to make all of them to be of same length.**

In [8]:
# now to create embeddings all of our docs need to be of same length. hence we can pad the docs with zeros.
pad_corp=pad_sequences(encod_corp,maxlen=maxlen,padding='post',value=0.0)
print("No of padded documents: ",len(pad_corp))

No of padded documents:  3


In [9]:
for i,doc in enumerate(pad_corp):
     print("The padded encoding for document",i+1," is : ",doc)

The padded encoding for document 1  is :  [ 5 14 17  6 35 18  0  0  0  0  0  0]
The padded encoding for document 2  is :  [46 48  6 35 18  6 17  6 47  0  0  0]
The padded encoding for document 3  is :  [49 20 14 17 33 18 28  2 48 47 18 33]


In [0]:
vocab_size=vocab_size
embed_size=100
max_doc_len=maxlen

#### CREATING AND BUILDING THE EMBEDDINGS AND TEHN LSTM MODEL

In [0]:
model=Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=embed_size,input_length=max_doc_len))
model.add(CuDNNLSTM(64,return_sequences=False))
model.add(Dense(32,activation='relu'))
model.add(Dense(2,activation='softmax'))

 

#### BREAKING IT DOWN ---

1) sabse pehle hum apne text ya reviews ki embeddings banayenge. refer keras ki embedding layer ki explanation agar koi doubt hora hai. is layer ka output jo hai uski shape jo hai vo (no of samples ,padded lenght, embedding size hoga).  for eg is sentiment analysis ke case me embedding layer ka output jo hai uska shape (no of reviews,no of words,embedding size) hoga.

2) ab next step hai is embedding ko lstm me sai tareeka se feed karna. First of all normal LSTM layer keras ki even Kaggle or Google cloud ke gpu pe bhut zyada slow hai isliye use CuDNNLSTM jo ki khali gpu ke liye hai.

**ab aati hai baari lstm ki layers ke input aur output or parametsr ki :**

**A) units : ** units jo hai vo output jo niklega lstm se uska dimension hai. actually agar week 1 of NG ke dl wale course se padoge to ye lstm la c(t) ya a(t) ka size hai. (size or number of hidden states or memory cell state).kai jagah like famous Colah ke blog me isse h(t) se denote kiya hai. for eg agar h(t) ek vector hhai to us vector ka size ya dimensions ye unit parameter hai. nutshell jo har timestep pe cell state(c(t)) ya output(h(t)) hai uska dimension hai ye.

**NOTE THAT YE NO OF LSTM CELLS NAI HAI LIKE JAISE COLAH KE DIAGRAM ME JO CELLS BANE HAI YTA JO NG BANAT HAI YE VO NO OF CELLS NAI HAI BALKI OUTPUT DIMENSION HAI.**

**FOR BETTER INITUITION SEE MODEL SUMMARY.**


**B )return_sequences : ** ye jo hai keval tab chhaiye hota hai jab lstm layers stack karni hai ya lstm +gru. basically agar say har lstm cell se output chahiye like in machine translation (jaha jar cell ka output ekword hoga) ya in general jaha bhi output bhi ek sequence hai vaha chahiye hoga. kyunki yaha hume koi sequcne nai bas + ya _ chahiye isliye yaha iski zaroorat nai hai isliye false.

**AGAR YE FALSE HAI TO YE many-to-one lstm n/w hoga varna many-to-many.** MUST READ ANSWER ON SO **[this](https://stackoverflow.com/questions/38714959/understanding-keras-lstms)**.

**C ) JUST TO MENTION KUCH BLOGS PE input_shape BHI PARAMETER DIYA HOTA HAI lstm layer me jo mujhe to doc me dikhaaye nai diya. ig vo tab chahiye hoga jab input like x_train vagerah khud hi shape me hai and pehle embedding layer nai use karre hai. AND ISI ME AGAR input_shape=(None,) likh de to vo variable reviews ko bhi handle kar sakte hai but again ye cheez mujhe zyada clr nai hui**

**D )ITNA SAB LIKHNE KE BAAD BHI EK CHEEZ BACHHI VO HAI KI HUMNE NO OF TIMESTEPS YA NO OF WORDS (HAR TIME STEP PE EK WORD SEE NG SENTIMENT CLASSIFICATION VIDEO) VO KAHA SPECIFY KIYA HAI. TO JITNI MERI UNDERSTANDING HAI VO YE KEHTI HAI KI VO EMBEDDING LAYER KE OUTPUT SE HI UNDERSTOOD HAI. LIKE NO OF TIME-STEPS JO HAI VO NO OF WORDS HONGE AND HAR INPUT KA SIZE JO HAI (x(t)) in NG VO EMBEDDING KA SIZE YAANI HAR WORD KO JITNE ME EMBED KIYA AI USKASIZE HO JAAYEGA. THEREFORE RQED INPUT FOR LSTM IS **

**(samples,timesteps,features) which boils to (no of reviews,no of words (max_doc_len),embed _size) in our case.**

**TO CLARIFY THIS POINT MUST WATCH NG VIDEOS AND ESECIALLY VO SENTIMENT WAALI.**



3) NOW MOVING ON AB BASICALLY HAMARE PASS LSTM LAYER LAGAKE EK VECTOR AA JAAYEGA (h(t)) last lstm cell ka jo ab kisi bhi dense layer me feed kar sakte hai normal NN ki tarah jaaha hum ek vector NN me feed karte hai.



#### COMPILING AND FITTING THE MODEL AS ALWAYS

In [0]:
# compiling the model. parameters can be tuned as always.
model.compile(optimizer=keras.optimizers.Adam(lr=1e-3),loss='binary_crossentropy',metrics=['acc'])

In [14]:
model.summary() # summary of the model.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 12, 100)           5000      
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 64)                42496     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 49,642
Trainable params: 49,642
Non-trainable params: 0
_________________________________________________________________


**dhyaan se har layer ka output and input vagerah ka shape dekhna.**

In [0]:
batch_size=128
epochs=5

Y=[0,1,1] # just random kuch matlab nai hai bas model fit karke dikhaana hai isliye kuch le liya
Y=to_categorical(Y)
x_train,x_test,y_train,y_test=train_test_split(np.array(pad_corp),Y,test_size=0.20,random_state=42)

In [18]:
model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_test,y_test))

Train on 2 samples, validate on 1 samples
Epoch 1/5
2/2 [==============================] - 2s 820ms/step - loss: 0.6924 - acc: 0.5000 - val_loss: 0.7034 - val_acc: 0.0000e+00
Epoch 2/5
2/2 [==============================] - 0s 6ms/step - loss: 0.6817 - acc: 1.0000 - val_loss: 0.7168 - val_acc: 0.0000e+00
Epoch 3/5
2/2 [==============================] - 0s 4ms/step - loss: 0.6706 - acc: 1.0000 - val_loss: 0.7294 - val_acc: 0.0000e+00
Epoch 4/5
2/2 [==============================] - 0s 6ms/step - loss: 0.6593 - acc: 1.0000 - val_loss: 0.7425 - val_acc: 0.0000e+00
Epoch 5/5
2/2 [==============================] - 0s 5ms/step - loss: 0.6455 - acc: 1.0000 - val_loss: 0.7593 - val_acc: 0.0000e+00


#### BONUS::

just ek aur tarika hai model banane ka using keras functional API and not the Sequential wala tarika.

In [ ]:
inp=Input(shape=(samples,max_doc_len)) # None then shape is inferred from input.
emb=Embedding(input_dim=vocab_size,output_dim=embed_size,input_length=max_doc_len)(inp)
# input_length hata bhi sakte yaha. read docs carefully
lstm=CuDNNLSTM(32)(emb)
# ab normal dense layers
out=Dense(16,activation='relu')
out=Dense(2,activaftion='sigmoid') # final layer binary classification.

# bas iske baad fir voi compile karke normal fit lagake train vagerah kar sakte hai.